In [ ]:
green_areas = r'D:\osm2lulc01\uatlas\forest_and_evu.shp'

green_fid = 'ffif'

poly_evu = r'D:\osm2lulc01\osm2lulc01.gdb\osm\helping_poly_evu'

outfeat = r'D:\osm2lulc01\osm2lulc01.gdb\result_poly'

tmpws = r'D:\osm2lulc01\osm2lulc01.gdb\corre'

In [ ]:
import arcpy
import os

from gesri.dp import copy_feat
from gesri.rd.shp import shp_to_lyr
from gesri.gp.ovl import intersection
from gesri.gp.gen import dissolve
from gesri.tbl.joins import join_table
from gesri.tbl.attr import geom_attr_to_shp
from gesri.tbl.col import cols_calc

In [ ]:
srs = "PROJCS[\"ETRS_1989_Portugal_TM06\",GEOGCS[\"GCS_ETRS_1989\",DATUM[\"D_ETRS_1989\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Transverse_Mercator\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-8.133108333333334],PARAMETER[\"Scale_Factor\",1.0],PARAMETER[\"Latitude_Of_Origin\",39.66825833333333],UNIT[\"Meter\",1.0]]"

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
cp_green = copy_feat(green_areas, outfeat)

glyr = shp_to_lyr(cp_green)

d = {'evu' : shp_to_lyr(poly_evu)}

In [ ]:
for k in d:
    # Intersect
    ointersect = os.path.join(tmpws, f'pi_{k}')
    ilyr = intersection([glyr, d[k]], ointersect)
    
    # Dissolve
    odiss = os.path.join(tmpws, f'pidss_{k}')
    disslyr = dissolve(ilyr, odiss, green_fid)
    
    # Get Area
    disslyr = geom_attr_to_shp(
        disslyr, f"poly_{k}",
        geom_attr="AREA", srs=srs,
        area_unit="SQUARE_METERS"
    )
    
    # Join
    green_lyr = join_table(
        cp_green, disslyr,
        green_fid, green_fid,
        cols=[f"poly_{k}"]
    )
    
    # Fill null values
    green_lyr = cols_calc(green_lyr, f"poly_{k}", f"fillna(!poly_{k}!)", code="""def fillna(np):
        if not np:
            return 0
        
        return np"""
    )